In [19]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from copulas.multivariate.gaussian import GaussianMultivariate
from scipy.stats import t

In [3]:
# Load the daily prices and portfolio holdings data
prices_df = pd.read_csv('DailyPrices.csv')
holdings_df = pd.read_csv('Portfolio.csv')
prices_df.set_index('Date', inplace=True)
prices_df = prices_df.pct_change().dropna()

In [7]:
# Loop over each portfolio
for portfolio in holdings_df['Portfolio'].unique():
    # Subset the holdings data for the current portfolio
    stocks = holdings_df[holdings_df['Portfolio'] == portfolio]['Stock']
    holdings_subset = holdings_df[holdings_df['Portfolio'] == portfolio]
    holdings_subset.set_index('Stock', inplace=True)
    holdings_subset = holdings_subset.T
    portfolio_ = pd.DataFrame()
    
    # Calculate the portfolio value for each day and store it
    for i in holdings_subset.columns:
        if i in prices_df.columns:
            portfolio_[i] = prices_df[i] * holdings_df[holdings_df['Stock'] == i]['Holding'].values
    
    total_value = portfolio_.sum(axis=1)
    weights_df = portfolio_.div(total_value, axis=0)
    
    

In [22]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt

# Load the daily prices and portfolio holdings data
prices_df = pd.read_csv('DailyPrices.csv')
holdings_df = pd.read_csv('Portfolio.csv')
prices_df.set_index('Date', inplace=True)

# Define a function to fit a Gaussian copula to the data
def fit_copula(portfolio):
    stocks = holdings_df[holdings_df['Portfolio'] == portfolio]['Stock']
    holdings_subset = holdings_df[holdings_df['Portfolio'] == portfolio]
    holdings_subset.set_index('Stock', inplace=True)
    holdings_subset = holdings_subset.T
    portfolio_ = pd.DataFrame()
    for i in holdings_subset.columns:
        if i in prices_df.columns:
            portfolio_[i] = prices_df[i] * holdings_df[holdings_df['Stock'] == i]['Holding'].values
    # Calculate the log returns of the portfolio
    log_returns = np.log(portfolio_ / portfolio_.shift(1)).dropna()
    # Calculate the sample correlation matrix of the log returns
    corr_matrix = log_returns.corr()
    # Calculate the sample covariance matrix of the log returns
    cov_matrix = log_returns.cov()
    # Estimate the parameters of the Gaussian copula
    norm_dist = stats.norm(0, 1)
    norm_scores = norm_dist.ppf(log_returns.rank()/(len(log_returns)+1))
    norm_scores = pd.DataFrame(norm_scores, index=log_returns.index, columns=log_returns.columns)
    norm_corr_matrix = np.corrcoef(norm_scores.T)
    upper_tri = np.triu_indices(len(stocks), k=1)
    norm_corr = norm_corr_matrix[upper_tri]
    norm_cov = cov_matrix.iloc[upper_tri]
    norm_params = stats.norm.fit(norm_scores)
    return stats.gaussian_kde(norm_corr, bw_method='silverman'), norm_params, norm_scores

# Define a function to simulate joint log returns from a Gaussian copula
def simulate_joint_log_returns(kde, norm_params, size):
    norm_dist = stats.norm(*norm_params)
    u = norm_dist.rvs(size=(len(kde), size))
    corr_samples = kde.resample(u)
    chol = np.linalg.cholesky(corr_samples)
    norm_samples = np.random.normal(0, 1, size=(len(kde), size))
    return chol @ norm_samples

# Define a function to calculate the portfolio log returns from the joint log returns
def calculate_portfolio_log_returns(portfolio, joint_log_returns):
    stocks = holdings_df[holdings_df['Portfolio'] == portfolio]['Stock']
    holdings_subset = holdings_df[holdings_df['Portfolio'] == portfolio]
    holdings_subset.set_index('Stock', inplace=True)
    holdings_subset = holdings_subset.T
    portfolio_ = pd.DataFrame()
    for i in holdings_subset.columns:
        if i in prices_df.columns:
            portfolio_[i] = prices_df[i] * holdings_df[holdings_df['Stock'] == i]['Holding'].values
    log_returns = np.log(portfolio_ / portfolio_.shift(1)).dropna()
    weights = holdings_subset.loc[stocks]['Holding'] / holdings_subset.loc[stocks]['Holding'].sum()
    portfolio_log_returns = (weights * (log_returns @ joint_log_returns.T)).sum(axis=1)


In [ ]:
# Define the Copula function using multivariate Gaussian
copula = GaussianMultivariate()

# Fit a multivariate Gaussian distribution to the stock returns
returns = np.log(prices_df / prices_df.shift(1)).dropna()
covariance = returns.cov().values
mean = returns.mean().values
copula.fit(returns)

# Perform joint simulation of the stock returns using the Copula function
simulated_returns = copula.simulate(n=10000)

# Calculate the simulated portfolio values for each day
simulated_values = {}
for i, portfolio in enumerate(portfolios):
    holdings_subset = holdings_df[holdings_df['Portfolio'] == portfolio]
    holdings_subset.set_index('Stock', inplace=True)
    holdings_subset = holdings_subset.T
    portfolio_ = pd.DataFrame()
    
    for j, stock in enumerate(holdings_subset.columns):
        if stock in prices_df.columns:
            portfolio_[stock] = simulated_returns[:, j+i*3] * holdings_subset[stock].values[0]
    
    simulated_values[portfolio] = portfolio_.sum(axis=1)

# Calculate the portfolio VaR and ES for each simulated portfolio value
VaR = {}
ES = {}
for portfolio in portfolios:
    portfolio_values = simulated_values[portfolio]
    portfolio_var = np.percentile(portfolio_values, 5)
    portfolio_es = -np.mean(portfolio_values[portfolio_values <= portfolio_var])
    VaR[portfolio] = portfolio_var
    ES[portfolio] = portfolio_es

# Calculate the total VaR and ES
total_var = sum(VaR.values())
total_es = sum(ES.values())

# Print the results
print(f"Portfolio A VaR: ${VaR['A']:,.2f}, ES: ${ES['A']:,.2f}")
print(f"Portfolio B VaR: ${VaR['B']:,.2f}, ES: ${ES['B']:,.2f}")
print(f"Portfolio C VaR: ${VaR['C']:,.2f}, ES: ${ES['C']:,.2f}")
print(f"Total VaR: ${total_var:,.2f}, ES: ${total_es:,.2f}")


/opt/anaconda3/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:639: RuntimeWarning: invalid value encountered in sqrt
  sk = 2*(b-a)*np.sqrt(a + b + 1) / (a + b + 2) / np.sqrt(a*b)
/opt/anaconda3/lib/python3.9/site-packages/scipy/optimize/minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
/opt/anaconda3/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:5320: RuntimeWarning: divide by zero encountered in true_divide
  return c**2 / (c**2 - n**2)
/opt/anaconda3/lib/python3.9/site-packages/scipy/stats/_distn_infrastructure.py:2606: RuntimeWarning: invalid value encountered in double_scalars
  Lhat = muhat - Shat*mu
/opt/anaconda3/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:5311: RuntimeWarning: divide by zero encountered in power
  return cd2*x**(c-1)
/opt/anaconda3/lib/python3.9/site-packages/copulas/univariate/truncated_

In [5]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
from copulas.multivariate import GaussianMultivariate
from copulas.univariate.gaussian import GeneralizedPareto

# Load the daily prices and portfolio holdings data
prices_df = pd.read_csv('DailyPrices.csv')
holdings_df = pd.read_csv('Portfolio.csv')
prices_df.set_index('Date', inplace=True)
prices_df = prices_df.pct_change().dropna()

# Fit a Generalized T distribution to each stock
gt_dists = {}
for stock in prices_df.columns:
    params = stats.genpareto.fit(prices_df[stock])
    gt_dist = GeneralizedPareto(*params)
    gt_dists[stock] = gt_dist

# Define the Copula function using Gaussian copula
copula = GaussianMultivariate()
copula.fit(prices_df)

# Perform joint simulation of the stock returns using the Copula function
simulated_returns = copula.sample(10000)
joint_log_returns = np.log(simulated_returns / simulated_returns.shift(1)).dropna().values

# Create an empty dictionary to store the portfolio dataframes
VaR = {}
ES = {}
total_var = 0
total_es = 0

# Loop over each portfolio
for portfolio in holdings_df['Portfolio'].unique():
    # Calculate the portfolio log returns
    portfolio_log_returns = calculate_portfolio_log_returns(portfolio, joint_log_returns)
    
    # Calculate the portfolio VaR and ES using the Generalized T distribution
    weights = holdings_df[holdings_df['Portfolio'] == portfolio].set_index('Stock')['Holding'] / holdings_df[holdings_df['Portfolio'] == portfolio]['Holding'].sum()
    var = np.abs(np.percentile(portfolio_log_returns, 5, axis=0))
    es = -np.mean(portfolio_log_returns[portfolio_log_returns <= -var])
    portfolio_var = (weights * var).sum()
    portfolio_es = (weights * es).sum()
    
    VaR[portfolio] = portfolio_var
    ES[portfolio] = portfolio_es
    
    print(f"Portfolio {portfolio} VaR: {portfolio_var:.2f}")
    print(f"Portfolio {portfolio} ES: {portfolio_es:.2f}")

    total_var += portfolio_var
    total_es += portfolio_es

print(f"Total VaR: {total_var:.2f}")
print(f"Total ES: {total_es:.2f}")


ImportError: cannot import name 'GeneralizedPareto' from 'copulas.univariate.gaussian' (/opt/anaconda3/lib/python3.9/site-packages/copulas/univariate/gaussian.py)

In [8]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
from copulas.multivariate import GaussianMultivariate
from scipy.stats import genpareto

# Load the daily prices and portfolio holdings data
prices_df = pd.read_csv('DailyPrices.csv')
holdings_df = pd.read_csv('Portfolio.csv')
prices_df.set_index('Date', inplace=True)
prices_df = prices_df.pct_change().dropna()

# Define the Copula function using Gaussian copula
copula = GaussianMultivariate()
copula.fit(prices_df)

# Loop over each portfolio
for portfolio in holdings_df['Portfolio'].unique():
    # Subset the holdings data for the current portfolio
    stocks = holdings_df[holdings_df['Portfolio'] == portfolio]['Stock']
    holdings_subset = holdings_df[holdings_df['Portfolio'] == portfolio]
    holdings_subset.set_index('Stock', inplace=True)
    holdings_subset = holdings_subset.T
    portfolio_ = pd.DataFrame()

    # Calculate the portfolio value for each day and store it
    for i in holdings_subset.columns:
        if i in prices_df.columns:
            portfolio_[i] = prices_df[i] * holdings_df[holdings_df['Stock'] == i]['Holding'].values

    # Fit a Generalized T model to each stock
    joint_log_returns = np.log(copula.sample(len(portfolio_)))
    stocks_log_returns = np.log(portfolio_ / portfolio_.shift(1)).dropna()
    stocks_returns = (stocks_log_returns @ joint_log_returns.T).T
    gts = []
    for i, stock in enumerate(stocks):
        gts.append(GeneralizedPareto())
        gts[i].fit(stocks_returns[:, i])

    # Calculate the portfolio log returns and weights
    log_returns = np.log(portfolio_ / portfolio_.shift(1)).dropna()
    weights = holdings_subset.loc[stocks]['Holding'] / holdings_subset.loc[stocks]['Holding'].sum()

    # Calculate the portfolio returns and VaR and ES using the Generalized T model
    portfolio_returns = (weights * (log_returns @ gts[0].sample(len(log_returns)).T)).sum(axis=1)
    var_gt = np.percentile(portfolio_returns, 5)
    es_gt = -(portfolio_returns[portfolio_returns <= var_gt].mean())

    # Print the results for the current portfolio
    print(f"Portfolio {portfolio}:")
    print(f"  VaR(0.05): {var_gt:.2f}")
    print(f"  ES(0.05): {es_gt:.2f}")

# Calculate the total portfolio returns and VaR and ES using the Generalized T model
weights = holdings_df.groupby('Stock')['Holding'].sum() / holdings_df['Holding'].sum()
portfolio_returns = np.zeros(len(prices_df))
for i, stock in enumerate(prices_df.columns):
    if stock in weights.index:
        returns = np.log(prices_df[stock] / prices_df[stock].shift(1)).dropna()
        portfolio_returns += weights[stock] * (returns @ gts[i].sample(len(returns)).T)
var_gt = np.percentile(portfolio_returns, 5)
es_gt = -(portfolio_returns[portfolio_returns <= var_gt].mean())

# Print the results for the total portfolio
print("Total Portfolio:")


/opt/anaconda3/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:639: RuntimeWarning: invalid value encountered in sqrt
  sk = 2*(b-a)*np.sqrt(a + b + 1) / (a + b + 2) / np.sqrt(a*b)
/opt/anaconda3/lib/python3.9/site-packages/scipy/optimize/minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
/opt/anaconda3/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:5320: RuntimeWarning: divide by zero encountered in true_divide
  return c**2 / (c**2 - n**2)
/opt/anaconda3/lib/python3.9/site-packages/scipy/stats/_distn_infrastructure.py:2606: RuntimeWarning: invalid value encountered in double_scalars
  Lhat = muhat - Shat*mu
/opt/anaconda3/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:5311: RuntimeWarning: divide by zero encountered in power
  return cd2*x**(c-1)
/opt/anaconda3/lib/python3.9/site-packages/copulas/univariate/truncated_

ValueError: matrices are not aligned

In [9]:
# Load the daily prices and portfolio holdings data
prices_df = pd.read_csv('DailyPrices.csv')
holdings_df = pd.read_csv('Portfolio.csv')
# Calculate daily log returns for each stock
log_returns = np.log(prices_df.set_index('Date') / prices_df.set_index('Date').shift(1)).dropna()

# Define the Copula function using Gaussian copula
copula = GaussianMultivariate()
copula.fit(log_returns)

In [10]:
# Perform joint simulation of the stock returns using the Copula function
n_simulations = 10000
simulated_returns = copula.sample(n_simulations)

In [20]:
# Fit a t distribution to the simulated returns of each stock
t_dists = {}
for stock in simulated_returns.columns:
    mu, std = simulated_returns[stock].mean(), simulated_returns[stock].std()
    df, _, _ = t.fit(simulated_returns[stock])
    t_dists[stock] = t(df, mu, std)

In [22]:
t_dists['SPY']

In [31]:
# Loop over each portfolio
for portfolio in holdings_df['Portfolio'].unique():
    # Subset the holdings data for the current portfolio
    stocks = holdings_df[holdings_df['Portfolio'] == portfolio]['Stock']
    holdings_subset = holdings_df[holdings_df['Portfolio'] == portfolio]
    holdings_subset.set_index('Stock', inplace=True)
    holdings_subset = holdings_subset.T
    portfolio_ = pd.DataFrame()
    # Calculate the portfolio value for each day and store it
    for i in holdings_subset.columns:
        if i in log_returns.columns:
            portfolio_[i] = np.exp(log_returns[i]) * holdings_df[holdings_df['Stock'] == i]['Holding'].values
    portfolio_values = portfolio_.sum(axis=1)
    # Calculate the daily log returns and weights for the portfolio
    log_returns_portfolio = np.log(portfolio_values / portfolio_values.shift(1)).dropna()
    weights = holdings_subset.T['Holding'] / holdings_subset.T['Holding'].sum()
    # Calculate the simulated portfolio returns
    simulated_portfolio_returns = (weights * (log_returns_portfolio @ simulated_returns.T)).values
    
    # Fit a t distribution to the simulated portfolio returns
    mu, std = simulated_portfolio_returns.mean(), simulated_portfolio_returns.std()
    df, _, _ = t.fit(simulated_portfolio_returns)
    portfolio_t_dist = t(df, mu, std)

ValueError: matrices are not aligned

In [37]:
from copulas.univariate import TruncatedGaussian

In [38]:
# Load the daily prices and portfolio holdings data
prices_df = pd.read_csv('DailyPrices.csv')
holdings_df = pd.read_csv('Portfolio.csv')
prices_df.set_index('Date', inplace=True)
prices_df = prices_df.pct_change().dropna()

# Fit a Generalized T model to each stock
models = {}
for stock in prices_df.columns:
    data = prices_df[stock].values
    model = TruncatedGaussian.from_samples(data, low=0)
    models[stock] = model

# Define the Copula function using Gaussian copula
copula = GaussianMultivariate(distribution="t")

# Perform joint simulation of the stock returns using the Copula function
simulated_returns = copula.sample(models, size=10000)


AttributeError: type object 'TruncatedGaussian' has no attribute 'from_samples'